In [1]:
from random import randint, uniform
from util import generate_problem, verify_witness

In [8]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def fitness(gene, T):
    # TODO
    # different ways of computing fitness
    # like: max distance for constraint to be satisfied
    #       number of unsatisfied constraints
    #       this, but weighted somehow, etc
    
    return -len(verify_witness(gene, T))

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]

def crossover(genes, gene_pool_size):
    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, gene_pool_size-2)
        genes += [ genes[i][cross_index:] + genes[j][:cross_index] ]
        
    return genes

def mutate(genes, mutation_chance, r):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
            g[randint(0, len(g)-1)] = randint(-r, r)
            
    return genes
        
def genetic(T, r,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T, r) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size)
        genes = mutate(genes, mutation_chance, r)
    
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = verify_witness(best_gene, T)
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [9]:
T = generate_problem(20, scaling_factor=1)
genetic(
    T=T, r=100,
    gene_pool_size=50,
    retainment_ratio=0.1,
    mutation_chance=0.1,
    max_iterations=2000,
)

best gene: [88, 85, 19, 3, 0, 90, 77, 78, 2, 20, 19, 16, 19, 50, 62, 74, 29, 35, 83, 78]
constraints failed: 3 out of: 59
failed constraints: [{'i': 2, 'j': 6, 'intervals': [(-89, -86), (-23, -14), (30, 31)]}, {'i': 4, 'j': 16, 'intervals': [(-92, -50), (-39, -36), (51, 71)]}, {'i': 7, 'j': 9, 'intervals': [(-2, 55)]}]
